In [3]:
import os
import itertools
import Levenshtein
from pyexpat import model
import pandas as pd
from tqdm import tqdm

seed = 0
method_list = ["max", "top_1%", "top_5%", "top_10%", "top_20%"]
# method = "top_1%"#max top_1%
top_model_name = "lin"
model_name = "eUniRep-Augmenting" # eUniRep-Augmenting_concate
train_num = 1000
des_method = "lim_pos_top_10-lim_dis_18"
do_method3 = 0
gfp_wt = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"
AA_LIST = ['A', 'C', 'D', 'E', 'F', 
            'G', 'H', 'I', 'K', 'L', 
            'M', 'N', 'P', 'Q', 'R', 
            'S', 'T', 'V', 'W', 'Y']
for method in method_list:
    
    input_path = f"/share/jake/github/low_n_output/method_2/{des_method}_3_mutation/{top_model_name}/{model_name}/{des_method}_{model_name}_{top_model_name}_random_1-2_train_num_{train_num}_do_method3_{do_method3}_seed_{seed}_{method}_func_result.csv"
    
    output_path = f"/share/jake/github/low_n_output/method_2/design_seqs/{top_model_name}/{model_name}_train_num_{train_num}_seed_{seed}_{method}_do_method3_{do_method3}_des_method_{des_method}"
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    top_hotspot_df = pd.read_csv(input_path)
    top_hotspot_name_list = list(top_hotspot_df["name"])

    def combination_3(combination_list):
        list_1 = []
        for chr_1 in combination_list:
            for chr_2 in combination_list:
                for chr_3 in combination_list:
                    list_1.append((chr_1, chr_2, chr_3))
        return list_1

    def saturation_mutationv(mutation_pos, template_seq):
        mut_pos_list = []
        for pos in mutation_pos.split():
            mut_pos_list.append(int(pos))
        mutation_list = []
        for amino_list in combination_3(AA_LIST):
            seq = list(template_seq)
            for num in range(len(amino_list)):
                # print(seq[mut_pos_list[num]])
                assert seq[mut_pos_list[num]] == "_"
                seq[mut_pos_list[num]] = amino_list[num]
            if Levenshtein.distance(gfp_wt, ''.join(seq)) == 3:
                mutation_list.append(''.join(seq))
        return mutation_list
    
    for name in tqdm(top_hotspot_name_list[0:10]):
        df_1 = top_hotspot_df[(top_hotspot_df["name"] == name)]
        template_seq = list(df_1["seqs"])[0]
        # print(template_seq)
        mutation_pos = name
        mutation_list = saturation_mutationv(mutation_pos, template_seq)
        template_seq_list = []
        name_list = []
        for i in range(len(mutation_list)):
            name_list.append(str(i))
            template_seq_list.append(template_seq)
        df_result = pd.DataFrame({"name": name_list, "seq": mutation_list, "template_seq": template_seq_list})
        df_result.to_csv(f"{output_path}/{name}.csv")
        # print(mutation_list[0])
        # print(len(mutation_list))

100%|██████████| 10/10 [00:01<00:00,  7.87it/s]


In [30]:
for name in tqdm(top_hotspot_name_list[0:20]):
    df_1 = top_hotspot_df[(top_hotspot_df["name"] == name)]
    template_seq = list(df_1["seqs"])[0]
    # print(template_seq)
    mutation_pos = name
    mutation_list = saturation_mutationv(mutation_pos, template_seq)
    template_seq_list = []
    name_list = []
    for i in range(len(mutation_list)):
        name_list.append(str(i))
        template_seq_list.append(template_seq)
    df_result = pd.DataFrame({"name": name_list, "seq": mutation_list, "template_seq": template_seq_list})
    df_result.to_csv(f"{output_path}/{name}.csv")
    # print(mutation_list[0])
    # print(len(mutation_list))

    

100%|██████████| 20/20 [00:03<00:00,  6.58it/s]


In [3]:
import os
import itertools
import pandas as pd
from tqdm import tqdm

gfp_wt = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"
AA_LIST = ['A', 'C', 'D', 'E', 'F', 
            'G', 'H', 'I', 'K', 'L', 
            'M', 'N', 'P', 'Q', 'R', 
            'S', 'T', 'V', 'W', 'Y']

target_name = "sk_10892"
all_sk_df = pd.read_csv("/share/jake/Low_N_data/csv/sk_data_set_distance.csv")
output_path = "/share/jake/hot_spot/data/test/temporary"
target_df = all_sk_df[(all_sk_df["name"] == target_name)]
target_seq = list(target_df["seq"])[0]
target_qfunc = list(target_df["quantitative_function"])[0]
print(target_df)

def combination_3(combination_list):
    list_1 = []
    for chr_1 in combination_list:
        for chr_2 in combination_list:
            for chr_3 in combination_list:
                list_1.append((chr_1, chr_2, chr_3))
    return list_1

def make_temple_seq(target_seq):
    seq = list(target_seq)
    wt_seq = list(gfp_wt)
    mutation_pos = []
    for i, mut in enumerate(seq):
        if wt_seq[i] != mut:
            mutation_pos.append(i)
            seq[i] = "_"
        else:
            assert wt_seq[i] == mut
    mutation_pos_str = ""
    for i in range(len(mutation_pos)):
        mutation_pos_str += str(mutation_pos[i])
        if i < len(mutation_pos) - 1:
            mutation_pos_str += " "
    return ''.join(seq), mutation_pos_str


def saturation_mutationv(mutation_pos, template_seq):
    mut_pos_list = []
    for pos in mutation_pos.split():
        mut_pos_list.append(int(pos))
    mutation_list = []
    for amino_list in combination_3(AA_LIST):
        seq = list(template_seq)
        for num in range(len(amino_list)):
            # print(seq[mut_pos_list[num]])
            assert seq[mut_pos_list[num]] == "_"
            seq[mut_pos_list[num]] = amino_list[num]
        mutation_list.append(''.join(seq))
    return mutation_list

template_seq, mutation_pos = make_temple_seq(target_seq)
mutation_list = saturation_mutationv(mutation_pos, template_seq)
template_seq_list = []
name_list = []
for i in range(len(mutation_list)):
    name_list.append(str(i))
    template_seq_list.append(template_seq)
df_result = pd.DataFrame({"name": name_list, "seq": mutation_list, "template_seq": template_seq_list})
df_oriange = df_result[(df_result["seq"] == target_seq)]
print(df_oriange)
df_result.to_csv(f"{output_path}/{mutation_pos}.csv")

       Unnamed: 0  Unnamed: 0.1      name  \
10892       10892         10892  sk_10892   

                                                     seq  \
10892  MSKGEELFTGVVPILVELDGDVNGQKFSVSGEGEGDATYGKLTLKF...   

       quantitative_function  distance  
10892               1.145045         3  
      name                                                seq  \
5516  5516  MSKGEELFTGVVPILVELDGDVNGQKFSVSGEGEGDATYGKLTLKF...   

                                           template_seq  
5516  MSKGEELFTGVVPILVELDGDVNG_KFSVSGEGEGDATYGKLTLKF...  
